# पाठ वर्गीकरण कार्य

जसरी हामीले उल्लेख गरेका छौं, हामी **AG_NEWS** डेटासेटमा आधारित साधारण पाठ वर्गीकरण कार्यमा केन्द्रित हुनेछौं, जसले समाचार शीर्षकहरूलाई चार श्रेणीहरूमा वर्गीकृत गर्नुपर्छ: विश्व, खेलकुद, व्यापार, र विज्ञान/प्रविधि।

## डेटासेट

यो डेटासेट [`torchtext`](https://github.com/pytorch/text) मोड्युलमा समावेश गरिएको छ, त्यसैले हामी यसलाई सजिलै पहुँच गर्न सक्छौं।


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

यहाँ, `train_dataset` र `test_dataset` ले क्रमशः वर्गको संख्या (लेबल) र पाठको जोडीहरू फर्काउने संग्रहहरू समावेश गर्दछ, उदाहरणका लागि:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

त्यसैले, हाम्रो डाटासेटबाट पहिलो १० नयाँ शीर्षकहरू प्रिन्ट गरौं:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

किनभने डेटासेटहरू इटरेटरहरू हुन्, यदि हामीले डाटा धेरै पटक प्रयोग गर्न चाहन्छौं भने हामीले यसलाई सूचीमा रूपान्तरण गर्न आवश्यक छ:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## टोकनाइजेशन

अब हामीले पाठलाई **संख्या** मा रूपान्तरण गर्न आवश्यक छ जसलाई टेन्सरको रूपमा प्रस्तुत गर्न सकिन्छ। यदि हामी शब्द-स्तरको प्रतिनिधित्व चाहन्छौं भने, हामीले दुई कुरा गर्नुपर्छ:
* पाठलाई **टोकन** मा विभाजन गर्न **टोकनाइजर** प्रयोग गर्नुहोस्
* ती टोकनहरूको **शब्दकोश** निर्माण गर्नुहोस्।


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

शब्दावली प्रयोग गरेर, हामी सजिलैसँग टोकन गरिएको स्ट्रिङलाई संख्याको सेटमा एन्कोड गर्न सक्छौं:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## शब्दहरूको झोला (Bag of Words) पाठ प्रतिनिधित्व

किनभने शब्दहरूले अर्थ व्यक्त गर्छन्, कहिलेकाहीँ हामी वाक्यको क्रमको पर्वाह नगरी, व्यक्तिगत शब्दहरू हेरेरै पाठको अर्थ पत्ता लगाउन सक्छौं। उदाहरणका लागि, समाचार वर्गीकरण गर्दा, *मौसम*, *हिउँ* जस्ता शब्दहरूले *मौसम पूर्वानुमान* जनाउन सक्छन्, जबकि *स्टक*, *डलर* जस्ता शब्दहरूले *आर्थिक समाचार* तर्फ सङ्केत गर्न सक्छन्।

**शब्दहरूको झोला** (BoW) भेक्टर प्रतिनिधित्व परम्परागत भेक्टर प्रतिनिधित्वमा सबैभन्दा धेरै प्रयोग गरिने विधि हो। प्रत्येक शब्दलाई भेक्टरको सूचकसँग जोडिन्छ, र भेक्टरको तत्त्वले कुनै विशेष दस्तावेजमा शब्दको उपस्थितिको सङ्ख्या समावेश गर्दछ।

![शब्दहरूको झोला भेक्टर प्रतिनिधित्व मेमोरीमा कसरी देखिन्छ भन्ने देखाउने छवि।](../../../../../translated_images/ne/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: तपाईं BoW लाई पाठका व्यक्तिगत शब्दहरूको लागि सबै एक-तर्फ-कोड गरिएको भेक्टरहरूको योगको रूपमा पनि सोच्न सक्नुहुन्छ।

तल Scikit Learn पायथन पुस्तकालय प्रयोग गरेर शब्दहरूको झोला प्रतिनिधित्व कसरी निर्माण गर्ने उदाहरण दिइएको छ:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS डेटासेटको भेक्टर प्रतिनिधित्वबाट bag-of-words भेक्टर गणना गर्न, हामी निम्न फङ्सन प्रयोग गर्न सक्छौं:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **नोट:** यहाँ हामीले डिफल्ट शब्दकोशको आकार निर्दिष्ट गर्न ग्लोबल `vocab_size` भेरिएबल प्रयोग गरेका छौं। किनभने प्रायः शब्दकोशको आकार धेरै ठूलो हुन्छ, हामी शब्दकोशको आकारलाई सबैभन्दा बारम्बार प्रयोग हुने शब्दहरूमा सीमित गर्न सक्छौं। `vocab_size` को मान घटाएर तलको कोड चलाउने प्रयास गर्नुहोस्, र यसले शुद्धतामा कसरी प्रभाव पार्छ हेर्नुहोस्। तपाईंले केही शुद्धता घट्ने अपेक्षा गर्नुपर्छ, तर प्रदर्शन सुधारको सट्टामा नाटकीय रूपमा होइन।


## BoW वर्गीकरणकर्ता प्रशिक्षण

अब हामीले हाम्रो पाठको Bag-of-Words प्रतिनिधित्व कसरी निर्माण गर्ने भनेर सिकिसकेपछि, यसमा आधारित भएर एक वर्गीकरणकर्ता प्रशिक्षण गरौं। पहिलो चरणमा, हामीले हाम्रो डेटासेटलाई यस्तो तरिकाले प्रशिक्षणका लागि रूपान्तरण गर्नुपर्छ, जसले गर्दा सबै स्थानगत भेक्टर प्रतिनिधित्वहरूलाई Bag-of-Words प्रतिनिधित्वमा परिवर्तन गरिन्छ। यो `bowify` फंक्शनलाई मानक torch `DataLoader` मा `collate_fn` प्यारामिटरको रूपमा पास गरेर हासिल गर्न सकिन्छ:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

अब हामी एउटा साधारण वर्गीकरण न्युरल नेटवर्क परिभाषित गरौं जसमा एउटा मात्र रेखीय तह हुन्छ। इनपुट भेक्टरको आकार `vocab_size` बराबर हुन्छ, र आउटपुट आकार कक्षाहरूको संख्या (4) सँग मेल खान्छ। किनभने हामी वर्गीकरण कार्य समाधान गर्दैछौं, अन्तिम सक्रियता प्रकार्य `LogSoftmax()` हो।


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

अब हामी मानक PyTorch प्रशिक्षण लूप परिभाषित गर्नेछौं। किनभने हाम्रो डेटासेट धेरै ठूलो छ, हाम्रो शिक्षण उद्देश्यको लागि हामी केवल एक इपोकको लागि प्रशिक्षण गर्नेछौं, र कहिलेकाहीँ इपोकभन्दा पनि कम (प्रशिक्षण सीमित गर्न `epoch_size` प्यारामिटर निर्दिष्ट गर्न सकिन्छ)। हामीले प्रशिक्षणको क्रममा संचित प्रशिक्षण शुद्धता पनि रिपोर्ट गर्नेछौं; रिपोर्ट गर्ने आवृत्ति `report_freq` प्यारामिटर प्रयोग गरेर निर्दिष्ट गरिन्छ।


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams र N-Grams

Bag of words विधिको एउटा सीमितता भनेको केही शब्दहरू बहु-शब्द अभिव्यक्तिहरूको हिस्सा हुन्छन्। उदाहरणका लागि, 'hot dog' शब्दको अर्थ 'hot' र 'dog' शब्दहरूको अन्य सन्दर्भमा भन्दा पूर्ण रूपमा फरक हुन्छ। यदि हामी 'hot' र 'dog' शब्दहरूलाई सधैं एउटै भेक्टरद्वारा प्रतिनिधित्व गर्छौं भने, यसले हाम्रो मोडेललाई भ्रमित गर्न सक्छ।

यस समस्यालाई समाधान गर्न, **N-gram प्रतिनिधित्वहरू** प्रायः कागजात वर्गीकरणका विधिहरूमा प्रयोग गरिन्छ, जहाँ प्रत्येक शब्द, दुई-शब्द (bi-word) वा तीन-शब्द (tri-word) को आवृत्ति classifiers प्रशिक्षण गर्नका लागि उपयोगी विशेषता हुन्छ। उदाहरणका लागि, bigram प्रतिनिधित्वमा, हामी मूल शब्दहरूको अतिरिक्त सबै शब्द जोडीहरूलाई पनि शब्दकोशमा थप्नेछौं।

तल Scikit Learn प्रयोग गरेर bigram bag of word प्रतिनिधित्व कसरी निर्माण गर्ने भन्ने उदाहरण छ:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram विधिको मुख्य कमजोरी भनेको शब्दकोशको आकार अत्यन्त छिटो बढ्न थाल्छ। व्यवहारमा, हामीलाई N-gram प्रतिनिधित्वलाई केही आयाम घटाउने प्रविधिहरूसँग संयोजन गर्न आवश्यक पर्छ, जस्तै *embeddings*, जसको बारेमा हामी अर्को इकाईमा छलफल गर्नेछौं।

हाम्रो **AG News** डेटासेटमा N-gram प्रतिनिधित्व प्रयोग गर्न, हामीले विशेष ngram शब्दकोश निर्माण गर्न आवश्यक छ:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


हामी माथिको जस्तै कोड प्रयोग गरेर classifier प्रशिक्षण गर्न सक्थ्यौं, तर यो धेरै मेमोरी-अक्षम हुने थियो। अर्को इकाईमा, हामी embeddings प्रयोग गरेर bigram classifier प्रशिक्षण गर्नेछौं।

> **Note:** तपाईंले केवल ती ngrams छोड्न सक्नुहुन्छ जुन पाठमा निर्दिष्ट गरिएको संख्याभन्दा बढी पटक देखा पर्छ। यसले सुनिश्चित गर्नेछ कि कम बारम्बार देखा पर्ने bigrams हटाइनेछ, र dimensionality उल्लेखनीय रूपमा घट्नेछ। यसका लागि, `min_freq` parameter लाई उच्च मानमा सेट गर्नुहोस्, र vocabulary को लम्बाइ परिवर्तनलाई अवलोकन गर्नुहोस्।


## टर्म फ्रिक्वेन्सी इनभर्स डकुमेन्ट फ्रिक्वेन्सी TF-IDF

BoW प्रतिनिधित्वमा, शब्दको उपस्थितिलाई समान तौल दिइन्छ, चाहे शब्द कस्तो भए पनि। तर, यो स्पष्ट छ कि बारम्बार प्रयोग हुने शब्दहरू, जस्तै *a*, *in*, आदि वर्गीकरणका लागि विशेष शब्दहरूको तुलनामा धेरै कम महत्त्वपूर्ण हुन्छन्। वास्तवमा, अधिकांश NLP कार्यहरूमा केही शब्दहरू अरूभन्दा बढी सान्दर्भिक हुन्छन्।

**TF-IDF** को अर्थ **टर्म फ्रिक्वेन्सी–इनभर्स डकुमेन्ट फ्रिक्वेन्सी** हो। यो बाग अफ वर्ड्सको एक भिन्नता हो, जहाँ कुनै शब्दको उपस्थितिलाई संकेत गर्ने 0/1 बाइनरी मानको सट्टा, फ्लोटिङ-पोइन्ट मान प्रयोग गरिन्छ, जुन शब्दको कर्पसमा उपस्थितिको फ्रिक्वेन्सीसँग सम्बन्धित हुन्छ।

औपचारिक रूपमा, शब्द $i$ को डकुमेन्ट $j$ मा तौल $w_{ij}$ यसरी परिभाषित गरिन्छ:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
जहाँ
* $tf_{ij}$ भनेको $j$ मा $i$ को उपस्थितिको संख्या हो, अर्थात् हामीले पहिले देखेको BoW मान
* $N$ भनेको संग्रहमा रहेका डकुमेन्टहरूको संख्या हो
* $df_i$ भनेको सम्पूर्ण संग्रहमा शब्द $i$ समावेश भएका डकुमेन्टहरूको संख्या हो

TF-IDF मान $w_{ij}$ शब्दको डकुमेन्टमा उपस्थितिको संख्याको अनुपातमा बढ्छ र कर्पसमा शब्द समावेश भएका डकुमेन्टहरूको संख्याले समायोजन गरिन्छ, जसले केही शब्द बारम्बार देखा पर्ने तथ्यलाई सम्हाल्न मद्दत गर्छ। उदाहरणका लागि, यदि शब्द *प्रत्येक* डकुमेन्टमा देखा पर्छ भने, $df_i=N$, र $w_{ij}=0$, र ती शब्दहरू पूर्ण रूपमा बेवास्ता गरिन्छन्।

तपाईं Scikit Learn प्रयोग गरेर सजिलै TF-IDF टेक्स्टको भेक्टराइजेसन बनाउन सक्नुहुन्छ:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## निष्कर्ष

यद्यपि TF-IDF प्रतिनिधित्वहरूले विभिन्न शब्दहरूलाई आवृत्तिको तौल प्रदान गर्छन्, तिनीहरूले अर्थ वा क्रमलाई प्रतिनिधित्व गर्न असमर्थ छन्। प्रसिद्ध भाषाविद् जे. आर. फर्थले १९३५ मा भनेका थिए, "शब्दको पूर्ण अर्थ सधैं सन्दर्भमा आधारित हुन्छ, र सन्दर्भबाहेकको अर्थको कुनै पनि अध्ययनलाई गम्भीरतापूर्वक लिन सकिँदैन।"। हामी यस पाठ्यक्रममा पछि पाठबाट भाषागत मोडेलिङ प्रयोग गरी कसरी पाठबाट सन्दर्भगत जानकारी कब्जा गर्ने भन्ने कुरा सिक्नेछौं।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी यथासम्भव शुद्धता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
